In [1]:
import concurrent.futures
import csv
import requests
from tqdm import tqdm
import json
from concurrent.futures import ThreadPoolExecutor,as_completed
from queue import Queue



def call_large_model_api(user_prompt, user_input):
    url = 'https://u21829-a763-f63ca723.westc.gpuhub.com:8443/v1/chat/completions'  # 请替换为实际的API URL
    headers = {
        'Content-Type': 'application/json',
        # 请替换为实际的API密钥
    }
    data = {
        "model": "llama3",
        "stream": False,
        "temperature": 0.01,
        "top_p": 0.8,
        "do_sample": True,
        "messages": [
            {
                "role": "system",
                "content": user_prompt
            },
            {
                "role": "user",
                "content": user_input
            }
        ]
    }
    
    response = requests.post(url, json=data, headers=headers)
    
    return response





# 处理每一行的函数
def process_row(row, result_queue):
    try:
        json_data = {}
        # 调用大模型API提取技术栈
        prompt = f'我会给你一个json,例如："职位名称_1":["数量": num,"技能":["技能1": skill_1, "重要性": times]]。然后给你一个 职位名称 + 职位描述 + 技能栈 。你需要把这个增添到我的json中，如果json中已经有相似的职位，则把 职位名称对应的数量+1，并且更新该职位中的技能，没有的置为1，有的就+1。最后只给我返回我需要的json ,不得生成其他文字。\n职位名称：{row[0]}, 职位描述: {row[12]},技能栈：{row[18]}'
        json_string = json.dumps(json_data)
        
        response = call_large_model_api(prompt, json_string)
        skill = response.json()
        result_queue.put([skill["choices"][0]["message"]["content"]])
        
    except Exception as e:
        print(f"Error processing row: {e}")

# 主函数
def main():
    # 初始化队列
    result_queue = Queue()
    info = []

    # 打开输入文件
    with open('Boss直聘/Boss直聘_skills.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # 跳过标题行

        # 使用线程池并行处理数据
        with ThreadPoolExecutor(max_workers=80) as executor:
            # 提交所有任务
            futures = [executor.submit(process_row, row, result_queue) for row in reader]

            # 遍历future对象，获取结果
            for future in tqdm(as_completed(futures), total=len(futures), desc="Processing rows", unit="row"):
                # 等待任务完成并获取结果
                info.append(result_queue.get())
                
    print(info)
# 运行主函数
if __name__ == "__main__":
    main()


Processing rows: 100%|██████████| 1000/1000 [11:30<00:00,  1.45row/s]

[['\n{"职位名称_1":["数量": 1, "技能": {"技能1": {"重要性": 1}}]}'], ['\n{"金蝶.NET开发工程师":["数量": 1, "技能": {"技能1": {"重要性": 1}}]}'], ['\n{"职位名称_1": [{"数量": 1, "技能": {"技能1": {"重要性": 1}}}]}'], ['\n{"职位名称_1": [{"数量": 1, "技能": {"技能1": {"技能": "任职资格", "重要性": 1}}}]}'], ['\n{"职位名称_1": [{"数量": 1, "技能": {"SQL": 1, "Hive": 1, "Hadoop生态圈": 1}}]}'], ['\n{"职位名称_1": [{"数量": 1, "技能": {"技能1": 1, "技能2": 1, "技能3": 1, "技能4": 1, "技能5": 1}}]}'], ['\n{"职位名称_1": [{"数量": 1, "技能": {"技能1": 1, "技能2": 1, "技能3": 1, "技能4": 1, "技能5": 1, "技能6": 1}}]}'], ['\n```json\n{\n  ".net开发工程师": {\n    "数量": 1,\n    "技能": {\n      "C#": 1,\n      "SQL Server数据库": 1,\n      "jQuery": 1,\n      "Bootstrap": 1\n    }\n  }\n}\n```'], ['\n```json\n{\n  ".NET 工程师": {\n    "数量": 2,\n    "技能": {\n      "Asp.net": 2,\n      "C#": 2,\n      "SQL": 2,\n      "英文文档读写": 2,\n      "团队合作精神": 2,\n      "责任感": 2\n    }\n  }\n}\n```'], ['\n```json\n{\n  ".net开发工程师": {\n    "数量": 2,\n    "技能": {\n      "DotNET开发": 4,\n      "SQLServer数据库": 3,\n      "多线程": 1,\n    

In [ ]:
import json
import pandas as pd
import re
import concurrent.futures

# 假设这是你的CSV文件路径
csv_file_path = 'Boss直聘/Boss直聘_skills.csv'

# 读取CSV文件
df = pd.read_csv(csv_file_path)


In [ ]:
data = df['岗位描述']
